### Clean Merge Result

In [ ]:
# !pip install modin[ray]

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 
# Not: Eğer multiprocessing kullanılacaksa ray init kullanılmamalı

In [ ]:
#import modin.pandas as pd
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 50)

#### Clean One Character And Digit

In [ ]:
df = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df

In [ ]:
def clean_num(var):
    var_list = re.findall("\w+", var)
    try:
        if var_list[0][0].isdigit() or (len(var_list) <= 1 and len(var_list[0]) <= 1):
            return "digit or one character"
        else:
            return var
    except:
        return "not digit or character"

In [ ]:
df.iloc[:,0] = df.iloc[:,0].apply(lambda x : clean_num(x))
df

In [ ]:
df[df.iloc[:,0] == "digit or one character"]

In [ ]:
df[df.iloc[:,0] == "not digit or character"]

In [ ]:
df.drop(df[df.iloc[:,0] == "digit or one character"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# df.drop(df[df.iloc[:,0] == "not digit or character"].index, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df

In [ ]:
df.to_csv("Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv", index=False)

#### Include Name Entity

In [ ]:
df_no_digit = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv")
df_no_digit = df_no_digit[["sentence"]].head(1000000)
df_no_digit

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [ ]:
help(multiprocessing)

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
df_ne = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx")
df_ne = df_ne.head(5)
df_ne

In [ ]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
special_name_list = manager.list()

def find_special_name(name_ent):
    name_in_df = df_no_digit[df_no_digit.iloc[:,0].str.contains(fr"(?:\s|^){name_ent}(?:\s|$)", na=True)]
    ne_list = name_in_df.iloc[:,0].to_list()
    for j in ne_list:
        special_name_list.append(j) 

if __name__ == '__main__':
    with Pool(8) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(find_special_name, ne_list) # string_word liste

In [ ]:
list_special_name = list(special_name_list)
df_special_name = pd.DataFrame(list_special_name)
df_special_name.rename(columns={0:"name_ent_include"}, inplace=True)
df_special_name

In [ ]:
df_special_name.to_csv("Name_Entity_Sentence.csv", index=False)